In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from traceback import format_exc
from pprint import pprint
import the_networks_of_war_python_functions

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
csv_directory = '/Users/the_networks_of_war/data_sources/csvs/'
pickle_directory = '/Users/the_networks_of_war/data_sources/pickles/'

# Descriptive Statistics for Each Country by Year
### Note: Applies to states/countries only.
### This will be joined to the participants of each war

### First, defining the participants by year to exclude any countries/years that will not be needed later on.

In [4]:
## unioning start and end years since that's how participant data will be joined in step 3
initial_part_df_1 = pd.read_pickle(pickle_directory + 'initial_part_df.pkl')[['c_code', 'start_year']]
initial_part_df_1.rename({'start_year': 'year'}, axis=1, inplace=True)
## year cannot be null
initial_part_df_1 = deepcopy(initial_part_df_1[initial_part_df_1['year'].isnull()==False])
initial_part_df_2 = pd.read_pickle(pickle_directory + 'initial_part_df.pkl')[['c_code', 'end_year']]
initial_part_df_2.rename({'end_year': 'year'}, axis=1, inplace=True)
## year cannot be null
initial_part_df_2 = deepcopy(initial_part_df_2[initial_part_df_2['year'].isnull()==False])

initial_part_df = deepcopy(pd.concat([initial_part_df_1, initial_part_df_2], sort=True, ignore_index=True).reset_index(drop=True))

initial_part_columns = ['c_code', 'year']
initial_part_df.drop_duplicates(subset=initial_part_columns, keep='first', inplace = True)
print('Total Participants in Initial Data: {}'.format(format(len(initial_part_df), ',d')))

Total Participants in Initial Data: 1,621


In [5]:
part_df_1 = pd.read_csv(csv_directory + 'alliance_v4.1_by_member_yearly.csv', encoding = 'latin-1')
part_df_1.rename({'ccode': 'c_code',
                  'defense': 'defense_alliances',
                  'entente': 'entente_alliances',
                  'neutrality': 'neutrality_alliances',
                  'ss_type': 'alliances'}, axis = 1, inplace = True)

## inner join to only include participants found in participants war data
## this will limit runtime significantly
part_df_1 = deepcopy(pd.merge(initial_part_df, part_df_1, how='inner', on=['c_code', 'year']))
    
part_df_1['defense_alliances'] = part_df_1['defense_alliances'].astype(float)
part_df_1['entente_alliances'] = part_df_1['entente_alliances'].astype(float)
part_df_1['neutrality_alliances'] = part_df_1['neutrality_alliances'].astype(float)

aggregations = {
    'alliances': 'count',
    'defense_alliances': 'sum',
    'entente_alliances': 'sum',
    'neutrality_alliances': 'sum',
    }
part_df_1 = deepcopy(part_df_1.groupby(['c_code', 'year']).agg(aggregations).reset_index())
## removing any duplicates that may have occured
part_df_1.drop_duplicates(subset=['c_code', 'year'], keep='first', inplace = True)

In [6]:
## dyadic trade data that will need to be adjusted to be non-dyadic (by country, by year)
part_df_2_1 = pd.read_csv(csv_directory + 'Dyadic_COW_4.0.csv', encoding='utf8')
part_df_2_1.rename({'ccode1': 'c_code_a',
                    'ccode2': 'c_code_b',
                    'flow2': 'money_flow_in_a',
                    ## money flow out
                    'flow1': 'money_flow_in_b'}, axis = 1, inplace = True)

## need to union to take summations but won't need to dedupe because there are no duplicates between a and b.
# this means a can be summed on its own when it's combined with b.
switched_columns_list = ['c_code_a',
                         'c_code_b',
                         'money_flow_in_a',
                         'money_flow_in_b']
part_df_2_1 = deepcopy(the_networks_of_war_python_functions.union_opposite_columns(part_df_2_1, switched_columns_list))
part_df_2_1.rename({'money_flow_in_a': 'money_flow_in',
                    'money_flow_in_b': 'money_flow_out'}, axis = 1, inplace = True)

aggregations = {'money_flow_in': 'sum',
                'money_flow_out': 'sum'}
part_df_2_1 = part_df_2_1.groupby(['c_code_a', 'year']).agg(aggregations).reset_index()
part_df_2_1.rename({'c_code_a':'c_code'}, axis = 1, inplace = True)

part_df_2_2 = pd.read_csv(csv_directory + 'National_COW_4.0.csv', encoding = 'latin-1')
part_df_2_2.rename({'ccode': 'c_code'}, axis = 1, inplace = True)
part_df_2_2 = deepcopy(part_df_2_2[['c_code',
                                    'year',
                                    'imports',
                                    'exports']])

part_df_2 = deepcopy(pd.merge(part_df_2_1, part_df_2_2, how='outer', on=['c_code', 'year']))

## inner join to only include participants found in participants war data
## this will limit runtime significantly
part_df_2 = deepcopy(pd.merge(initial_part_df, part_df_2, how='inner', on=['c_code', 'year']))
## removing any duplicates that may have occured
part_df_2.drop_duplicates(subset=['c_code', 'year'], keep='first', inplace = True)

In [7]:
# trade_df.head(3)

In [8]:
part_df_3 = pd.read_csv(csv_directory + 'NMC_5_0-wsupplementary.csv', encoding = 'latin-1')
part_df_3.rename({'milex': 'military_expenditure',
                  'milper': 'military_personnel',
                  'irst': 'iron_steel_production',
                  'pec': 'prim_energy_consumption',
                  'tpop': 'total_population',
                  'upop': 'urban_population',
                  'upopgrowth': 'urban_pop_growth_rate',
                  'ccode': 'c_code',
                  'cinc': 'cinc_score'}, axis = 1, inplace = True)

part_df_3 = part_df_3.sort_values(by = 'year', ascending = True).reset_index(drop = True)
part_df_3 = deepcopy(part_df_3[['c_code',
                                'year',
                                'military_expenditure',
                                'military_personnel',
                                'prim_energy_consumption',
                                'iron_steel_production',
                                'total_population',
                                'urban_population',
                                'cinc_score']])

## inner join to only include participants found in participants war data
## this will limit runtime significantly
part_df_3 = deepcopy(pd.merge(initial_part_df, part_df_3, how='inner', on=['c_code', 'year']))
## removing any duplicates that may have occured
part_df_3.drop_duplicates(subset=['c_code', 'year'], keep='first', inplace = True)

In [9]:
# ## not sure that this one is worth the trouble because ccodes are not included
# part_df_4 = pd.read_csv(csv_directory + 'co-emissions-per-capita.csv', encoding = 'latin-1')

In [10]:
descriptive_df_1 = deepcopy(pd.merge(part_df_1, part_df_2, how='outer', on=['c_code', 'year']))
descriptive_df_1 = deepcopy(pd.merge(descriptive_df_1, part_df_3, how='outer', on=['c_code', 'year']))
descriptive_df_1['year'] = descriptive_df_1['year'].astype(float)

In [11]:
print('Total Rows of Descriptive Participant Data: {}'.format(format(len(descriptive_df_1), ',d')))
pickle_directory = '/Users/the_networks_of_war/data_sources/pickles/'
descriptive_df_1.to_pickle(pickle_directory + 'participant_descriptive_df.pkl')

Total Rows of Descriptive Participant Data: 1,424


# Descriptive Statistics for Each Dyad by Year
### Note: Applies to states/countries only.
### This will be joined to the dyadic pairs for each war

### First, defining the dyads by year to exclude any countries/years that will not be needed later on.

In [12]:
initial_dyad_columns = ['c_code_a', 'c_code_b', 'year']
initial_dyad_df = pd.read_pickle(pickle_directory + 'initial_dyad_df.pkl')[initial_dyad_columns]
initial_dyad_df.drop_duplicates(subset=initial_dyad_columns, keep='first', inplace = True)

print('Total Dyads in Initial Data: {}'.format(format(len(initial_dyad_df), ',d')))

Total Dyads in Initial Data: 2,591


### Correlates of War Descriptive Data

In [13]:
## lot's to use in this dataset so I'll start with the basics
data_source = csv_directory + 'tc2018.csv'
dy_df_1 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'gainer', 'loser', 'year', 'territory_exchange'))
## must be dyadic (two states per row)
dy_df_1 = deepcopy(dy_df_1[(dy_df_1['c_code_a']!=-8) & (dy_df_1['c_code_a']!=-9) & (dy_df_1['c_code_b']!=-8) & (dy_df_1['c_code_b']!=-9)])

## contiguity dataframe for states of colonial dependencies
data_source = csv_directory + 'contcold.csv'
dy_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'statelno', 'statehno', 'year', 'colonial_contiguity'))

data_source = csv_directory + 'contdird.csv'
dy_df_3 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'state1no', 'state2no', 'year', 'contiguity'))

data_source = csv_directory + 'alliance_v4.1_by_dyad_yearly.csv'
dy_df_4 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'ccode1', 'ccode2', 'year', 'alliance'))

data_source = csv_directory + 'DCAD-v1.0-dyadic.csv'
dy_df_5 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'ccode1', 'ccode2', 'year', 'defense_cooperation_agreements'))

## must have at least one joined by both states in order to be included
data_source = csv_directory + 'dyadic_formatv3.csv'
dy_df_6 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'ccode1', 'ccode2', 'year', 'inter_governmental_organizations'))

## this one needs to be filled since its only 5 years
data_source = csv_directory + 'Diplomatic_Exchange_2006v1.csv'
dy_df_7 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'ccode1', 'ccode2', 'year', 'diplomatic_exchange'))

data_source = csv_directory + 'Dyadic_COW_4.0.csv'
dy_df_8 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'ccode1', 'ccode2', 'year', 'trade_relations'))

In [14]:
# for year in np.arange(1800, 2020):
#     for row in dy_df_7['year']:
#         if len(dy_df_7[dy_df_7['year']==year])== 0:
#             temp_dyad_df = deepcopy(dy_df_7[dy_df_7['year']==year].reset_index())
#             for i, dyad in enumerate(temp_dyad_df['year']):
#                 dyad_df_length = deepcopy(len(dy_df_7))
#                 dy_df_7.loc[dyad_df_length, 'year'] = year
#                 dy_df_7.loc[dyad_df_length, 'c_code_a'] = temp_dyad_df.loc[i, 'c_code_a']
#                 dy_df_7.loc[dyad_df_length, 'c_code_b'] = temp_dyad_df.loc[i, 'c_code_b']
#         else:
#             current_year = year
            
# dy_df_7['diplomatic_exchange'] = 1
# print(len(dy_df_7))

### Non-Correlates of War Descriptive Data

### Setting up processing of 'ddrevisited_data_v1'

In [15]:
dd_df_1 = pd.read_csv(csv_directory + 'ddrevisited_data_v1.csv', encoding = 'latin-1')[['cowcode', 'year', 'emil', 'royal', 'comm', 'democracy', 'regime', 'collect', 'incumb', 'legselec']]
dd_df_1.rename({'cowcode': 'c_code',
                'emil': 'military_leader',
                'royal': 'royal_leader',
                'comm': 'communist_leader',
                'democracy': 'democratic_regime',
                'collect': 'collective_leadership',
                'regime': 'regime_type',
                'incumb': 'incumbent_type',
                'legselec': 'legislation_type'}, axis = 1, inplace = True)

dd_df_2 = pd.read_csv(csv_directory + 'ddrevisited_data_v1.csv', encoding = 'latin-1')[['cowcode2', 'year', 'emil', 'royal', 'comm', 'democracy', 'regime', 'collect', 'incumb', 'legselec']]
dd_df_2.rename({'cowcode2': 'c_code',
                'emil': 'military_leader',
                'royal': 'royal_leader',
                'comm': 'communist_leader',
                'democracy': 'democratic_regime',
                'collect': 'collective_leadership',
                'regime': 'regime_type',
                'incumb': 'incumbent_type',
                'legselec': 'legislation_type'}, axis = 1, inplace = True)
## unioning the two ccodes above so they can both be represented
## this will also allow for substates to be joined to the larger states
## this will need to be recognized later on to prevent from saying same leadership when it's the same leader
dd_df = deepcopy(pd.concat([dd_df_1, dd_df_2]))
## removing duplicates from concat
duplicate_list = ['c_code',
                  'military_leader',
                  'royal_leader',
                  'communist_leader',
                  'democratic_regime',
                  'regime_type',
                  'collective_leadership',
                  'incumbent_type',
                  'legislation_type',
                  'year']
dd_df.drop_duplicates(subset=duplicate_list, keep='first', inplace = True)

dd_df = deepcopy(pd.merge(dd_df, dd_df, how='left', on=['year']))
    
for column in dd_df.columns:
    if column[-2:]=='_x':
        dd_df.rename({column: column[:-2] + '_a'}, axis = 1, inplace = True)
    elif column[-2:]=='_y':
        dd_df.rename({column: column[:-2] + '_b'}, axis = 1, inplace = True)
    else:
        pass
    
dd_df = deepcopy(dd_df[dd_df['c_code_a']!=dd_df['c_code_b']])

## removing dyads from descriptive data that won't be needed
dd_df = deepcopy(the_networks_of_war_python_functions.remove_extra_dyads(initial_dyad_df, dd_df))
## removing any duplicates that may have occured
dd_df.drop_duplicates(subset=['c_code_a', 'c_code_b', 'year'], keep='first', inplace = True)

In [16]:
conditional_statement = (dd_df['military_leader_a']==dd_df['military_leader_b']) & (dd_df['communist_leader_a']==dd_df['communist_leader_b']) & (dd_df['royal_leader_a']==dd_df['royal_leader_b']) & (dd_df['democratic_regime_a']==dd_df['democratic_regime_b'])
dy_df_9 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'same_leader_type'))

conditional_statement = (dd_df['military_leader_a']==1) & (dd_df['military_leader_b']==1)
dy_df_10 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'both_military_leaders'))

conditional_statement = (dd_df['communist_leader_a']==1) & (dd_df['communist_leader_b']==1)
dy_df_11 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'both_communist_leaders'))

conditional_statement = (dd_df['royal_leader_a']==1) & (dd_df['royal_leader_b']==1)
dy_df_12 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'both_royal_leaders'))

conditional_statement = dd_df['regime_type_a']==dd_df['regime_type_b']
dy_df_13 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'same_regime_type'))

conditional_statement = (((dd_df['democratic_regime_a']==1) | (dd_df['regime_type_a']==0) | (dd_df['regime_type_a']==1) | (dd_df['regime_type_a']==2)) & ((dd_df['democratic_regime_b']==1) | (dd_df['regime_type_b']==0) | (dd_df['regime_type_b']==1) | (dd_df['regime_type_b']==2)))
dy_df_14 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'both_democratic_regimes'))

conditional_statement = ((dd_df['regime_type_a']==3) | (dd_df['regime_type_a']==4) | (dd_df['regime_type_a']==5)) & ((dd_df['regime_type_b']==3) | (dd_df['regime_type_b']==4) | (dd_df['regime_type_b']==5))
dy_df_15 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'both_dictatorships'))

conditional_statement = (dd_df['collective_leadership_a']==1) & (dd_df['collective_leadership_b']==1)
dy_df_16 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'both_collective_leadership'))

conditional_statement = dd_df['incumbent_type_a']==dd_df['incumbent_type_b']
dy_df_17 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'same_incumbent_type'))

conditional_statement = dd_df['legislation_type_a']==dd_df['legislation_type_b']
dy_df_18 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_dd(dd_df, conditional_statement, 'same_legislation_type'))

In [17]:
# Alliance Treaty Obligations and Provisions (ATOP)
data_source = csv_directory + 'atop4_01ddyr.csv'
dy_df_19 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'stateA', 'stateB', 'year', 'atop'))

In [18]:
# The ICOW multilateral treaties of pacific settlement (MTOPS)
data_source = csv_directory + 'mtopsd150.csv'
dy_df_20 = pd.read_csv(data_source, encoding='utf8')
dy_df_20['total'] = dy_df_20['pacsettg'] + dy_df_20['pacsettr'] + dy_df_20['pacsett'] + dy_df_20['tergen'] + dy_df_20['terviol'] + dy_df_20['tertot']
dy_df_20 = deepcopy(dy_df_20[dy_df_20['total']>0])
dy_df_20 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, None, dy_df_20, 'state1', 'state2', 'year', 'mtops'))

In [19]:
# data_source = csv_directory + 'ucdp-peace-agreements-191.csv'
# dy_df_19 = pd.read_csv(data_source, encoding='utf8')

In [20]:
# # UCDP Conflict Termination Dataset version 2-2015
# data_source = csv_directory + 'ucdp-term-dyadic-2015.csv'
# dy_df_20 = pd.read_csv(data_source, encoding='utf8')

In [21]:
# # UCDP Battle-Related Deaths Dataset version 20.1
# data_source = csv_directory + 'UCDP Battle-Related Deaths Dataset version 20.1'
# dy_df_21 = pd.read_csv(data_source, encoding='utf8')

In [22]:
# # UCDP Actor Dataset version 20.1
# data_source = csv_directory + 'ucdp-actor-201.csv'
# dy_df_22 = pd.read_csv(data_source, encoding='utf8')

In [23]:
# # UCDP Non-state Conflict Issues and Actors Dataset
# data_source = csv_directory + 'UCDP_NS_IAD.csv'
# dy_df_23 = pd.read_csv(data_source, encoding='utf8')

In [24]:
# # UCDP External Support in Non-state Conflict Dataset
# data_source = csv_directory + 'UCDP External Support in Non-state Conflict Dataset v.1.0.csv'
# dy_df_24 = pd.read_csv(data_source, encoding='utf8')

In [25]:
# # UCDP Managing Intrastate Low-intensity Conflict (MILC) dataset
# data_source = csv_directory + 'milc-10.csv'
# dy_df_25 = pd.read_csv(data_source, encoding='utf8')

In [26]:
# # UCDP Managing Intrastate Conflict (MIC) dataset
# ## more than 2 ids per conflict
# data_source = csv_directory + 'micFINAL.csv'
# dy_df_26 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'contry_id', 'thirdid1', 'year', 'atop'))

In [27]:
# # 'gwno'
# # 'dyad_id'
# ## only including failed peace agreements
# dy_df_19 = deepcopy(dy_df_19[dy_df_19['ended']==True])[['gwno', 'dyad_id', 'dyad_name', 'actor_id', 'actor_name', 'year', 'duration', 'c_duration']]


In [28]:
descriptive_df_2 = deepcopy(pd.merge(dy_df_1, dy_df_2, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_3, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_4, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_5, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_6, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_7, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_8, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_9, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_10, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_11, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_12, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_13, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_14, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_15, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_16, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_17, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_18, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_19, how='outer', on=['c_code_a', 'c_code_b', 'year']))
descriptive_df_2 = deepcopy(pd.merge(descriptive_df_2, dy_df_20, how='outer', on=['c_code_a', 'c_code_b', 'year']))

In [29]:
print('Total Rows of Descriptive Dyadic Data: {}'.format(format(len(descriptive_df_2), ',d')))
descriptive_df_2.to_pickle(pickle_directory + 'dyadic_descriptive_df.pkl')

Total Rows of Descriptive Dyadic Data: 1,362
